In [20]:
%autoreload 2

In [21]:
from argparse import Namespace
from collections import defaultdict
import copy
import difflib
import gzip
import itertools
import os
import pickle
import sys
import typing

from IPython.display import display, Markdown, HTML
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from pactools.grid_search import GridSearchCVProgressBar
import pandas as pd
import sklearn
from sklearn.model_selection import GridSearchCV, train_test_split, KFold
from sklearn.pipeline import Pipeline
import tatsu
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import tqdm.notebook as tqdm


sys.path.append(os.path.abspath('..'))
sys.path.append(os.path.abspath('../src'))
from src import fitness_energy_utils as utils
from src.fitness_energy_utils import NON_FEATURE_COLUMNS
from src.ast_counter_sampler import *
from src.ast_utils import cached_load_and_parse_games_from_file, load_games_from_file, _extract_game_id
from src import ast_printer

In [22]:
fitness_df = utils.load_fitness_data()
print(fitness_df.src_file.unique())
fitness_df.head()

['interactive-beta.pddl' 'ast-real-regrowth-samples.pddl']


,Index,src_file,game_name,domain_name,all_variables_defined,all_variables_used,all_preferences_used,setup_objects_used,no_adjacent_once,starts_and_ends_once,...,node_count_terminal_2,node_count_terminal_3,node_count_terminal_4,node_count_scoring_0,node_count_scoring_1,node_count_scoring_2,node_count_scoring_3,node_count_scoring_4,real,original_game_name
0,0,interactive-beta.pddl,6172feb1665491d1efbce164-0,medium-objects-room-v1,1.0,1.0,1.0,1.00,1.0,0.5,...,1,0,0,1,0,0,0,0,True,6172feb1665491d1efbce164-0
1,1,interactive-beta.pddl,5f77754ba932fb2c4ba181d8-2,many-objects-room-v1,1.0,1.0,1.0,1.00,1.0,1.0,...,1,0,0,0,0,0,1,0,True,5f77754ba932fb2c4ba181d8-2
2,2,interactive-beta.pddl,614b603d4da88384282967a7-3,many-objects-room-v1,1.0,1.0,1.0,0.00,1.0,0.0,...,0,0,0,0,1,0,0,0,True,614b603d4da88384282967a7-3
3,3,interactive-beta.pddl,5bc79f652885710001a0e82a-5,few-objects-room-v1,1.0,1.0,1.0,0.00,1.0,1.0,...,0,0,0,1,0,0,0,0,True,5bc79f652885710001a0e82a-5
4,4,interactive-beta.pddl,614dec67f6eb129c3a77defd-6,medium-objects-room-v1,1.0,1.0,1.0,0.25,1.0,1.0,...,0,0,0,0,0,1,0,0,True,614dec67f6eb129c3a77defd-6


In [23]:
def create_filtered_df(df: pd.DataFrame, 
    synthetic_data_src_files: typing.Sequence[str] = ('interactive-beta.pddl', 'ast-real-regrowth-samples.pddl'),
    ) -> pd.DataFrame:
    syntethic_df = fitness_df[fitness_df.src_file.isin(synthetic_data_src_files)].reset_index(drop=True)
    syntethic_df.loc[syntethic_df.src_file == synthetic_data_src_files[0], 'real'] = 1
    return syntethic_df

filtered_fitness_df = create_filtered_df(fitness_df)

In [24]:
filtered_fitness_df.head()

,Index,src_file,game_name,domain_name,all_variables_defined,all_variables_used,all_preferences_used,setup_objects_used,no_adjacent_once,starts_and_ends_once,...,node_count_terminal_2,node_count_terminal_3,node_count_terminal_4,node_count_scoring_0,node_count_scoring_1,node_count_scoring_2,node_count_scoring_3,node_count_scoring_4,real,original_game_name
0,0,interactive-beta.pddl,6172feb1665491d1efbce164-0,medium-objects-room-v1,1.0,1.0,1.0,1.00,1.0,0.5,...,1,0,0,1,0,0,0,0,1,6172feb1665491d1efbce164-0
1,1,interactive-beta.pddl,5f77754ba932fb2c4ba181d8-2,many-objects-room-v1,1.0,1.0,1.0,1.00,1.0,1.0,...,1,0,0,0,0,0,1,0,1,5f77754ba932fb2c4ba181d8-2
2,2,interactive-beta.pddl,614b603d4da88384282967a7-3,many-objects-room-v1,1.0,1.0,1.0,0.00,1.0,0.0,...,0,0,0,0,1,0,0,0,1,614b603d4da88384282967a7-3
3,3,interactive-beta.pddl,5bc79f652885710001a0e82a-5,few-objects-room-v1,1.0,1.0,1.0,0.00,1.0,1.0,...,0,0,0,1,0,0,0,0,1,5bc79f652885710001a0e82a-5
4,4,interactive-beta.pddl,614dec67f6eb129c3a77defd-6,medium-objects-room-v1,1.0,1.0,1.0,0.25,1.0,1.0,...,0,0,0,0,0,1,0,0,1,614dec67f6eb129c3a77defd-6


In [25]:
fake_games = filtered_fitness_df[filtered_fitness_df.src_file == 'ast-real-regrowth-samples.pddl'].reset_index()
broadcasted_original = filtered_fitness_df.loc[[filtered_fitness_df.index[(filtered_fitness_df.game_name == original_name)][0] for original_name in fake_games.original_game_name], :].reset_index()

original_regrown_diffs = (broadcasted_original.drop(NON_FEATURE_COLUMNS, axis=1) - fake_games.drop(NON_FEATURE_COLUMNS, axis=1))

unchanged_games_prop = (original_regrown_diffs.drop('index', axis=1) == 0).all(axis=1).sum() / len(original_regrown_diffs)
print(f'In {unchanged_games_prop * 100:.2f}% of the games, the regrown game was identical to the original game.')

In 5.33% of the games, the regrown game was identical to the original game.


In [26]:
original_df = filtered_fitness_df[filtered_fitness_df.src_file == 'interactive-beta.pddl'].reset_index(drop=True)
arr = np.array(original_df.all_variables_used) 
ind = np.where(arr != 1)[0]
print(ind)
original_df.iloc[ind, :]

[]


,Index,src_file,game_name,domain_name,all_variables_defined,all_variables_used,all_preferences_used,setup_objects_used,no_adjacent_once,starts_and_ends_once,...,node_count_terminal_2,node_count_terminal_3,node_count_terminal_4,node_count_scoring_0,node_count_scoring_1,node_count_scoring_2,node_count_scoring_3,node_count_scoring_4,real,original_game_name


# Synthetic model-fitting experiment approach
We have a large dataset now, I can try to cross-validate over some of the choices I might make:
* Loss function
* Parameters of each loss function (margin, etc.)
* Batch size
* Number of negative examples sampled per iteration
* Regularization strength?
* ...?



In [33]:
def cross_validate(train: pd.DataFrame, feature_columns: typing.List[str],
    param_grid: typing.Union[typing.List[typing.Dict[str, typing.Any]], typing.Dict[str, typing.Any]],
    scoring_function: typing.Callable = utils.evaluate_fitness,
    model_kwargs: typing.Optional[typing.Dict[str, typing.Any]] = None,
    train_kwargs: typing.Optional[typing.Dict[str, typing.Any]] = None, 
    cv_kwargs: typing.Optional[typing.Dict[str, typing.Any]] = None,
    n_folds: int = 5, verbose: int = 0):

    if model_kwargs is None:
        model_kwargs = {}

    if train_kwargs is None:
        train_kwargs = {}

    if cv_kwargs is None:
        cv_kwargs = {}

    if 'n_jobs' not in cv_kwargs: 
        cv_kwargs['n_jobs'] = -1
    if 'verbose' not in cv_kwargs:
        cv_kwargs['verbose'] = verbose

    train_tensor = utils.df_to_tensor(train, feature_columns)
    pipeline = Pipeline(steps=[('scaler', utils.CustomSklearnScaler()), ('fitness', utils.SklearnFitnessWrapper(model_kwargs=model_kwargs, train_kwargs=train_kwargs))])

    if isinstance(param_grid, list):
        for param_grid_dict in param_grid:
            param_grid_dict['fitness__n_features'] = [len(feature_columns)]
    else:
        param_grid['fitness__n_features'] = [len(feature_columns)]        

    random_seed = train_kwargs['random_seed'] if 'random_seed' in train_kwargs else None

    cv = GridSearchCVProgressBar(pipeline, param_grid, scoring=scoring_function, 
        cv=KFold(n_folds, shuffle=True, random_state=random_seed), 
        **cv_kwargs)
    return cv.fit(train_tensor, None)


def model_fitting_experiment(df: pd.DataFrame,
    param_grid: typing.Union[typing.List[typing.Dict[str, typing.Any]], typing.Dict[str, typing.Any]], 
    feature_columns: typing.Optional[typing.List[str]] = None, 
    random_seed: int = utils.DEFAULT_RANDOM_SEED,
    scoring_function: typing.Callable = utils.evaluate_fitness,
    model_kwargs: typing.Optional[typing.Dict[str, typing.Any]] = None,
    train_kwargs: typing.Optional[typing.Dict[str, typing.Any]] = None,
    cv_kwargs: typing.Optional[typing.Dict[str, typing.Any]] = None,
    n_folds: int = 5, verbose: int = 0
    ):

    if model_kwargs is None:
        model_kwargs = {}

    if train_kwargs is None:
        train_kwargs = {}

    filtered_df = create_filtered_df(df)

    if feature_columns is None:
        feature_columns = [c for c in filtered_df.columns if c not in NON_FEATURE_COLUMNS]

    # rng = np.random.default_rng(random_seed)

    # train_df, test_df = utils.train_test_split_by_game_name(filtered_df, random_seed=random_seed)
    cv = cross_validate(filtered_df, feature_columns, param_grid, 
        scoring_function=scoring_function,
        train_kwargs={'random_seed': random_seed, **train_kwargs}, 
        model_kwargs=model_kwargs, cv_kwargs=cv_kwargs, n_folds=n_folds, verbose=verbose)
    # best_model = cv.best_estimator_.named_steps['fitness'].model  # type: ignore

    # TODO: add evaluation on the held-out part of the dataset here
    return cv, None



In [34]:
test_param_grid = [
    {
        'fitness__loss_function': [utils.fitness_hinge_loss], # [utils.fitness_hinge_loss_with_cross_example],
        'fitness__weight_decay': [0.0, 0.25, 1, 2],  
        'fitness__margin': [1, 2, 4, 8],
        'fitness__lr':  [1e-1, 3e-2, 1e-2, 3e-3],  # [1e-1, 3e-2, 1e-2, 3e-3],
        'fitness__k': [4, 8, 16, 32, 64],
        'fitness__batch_size': [2, 4, 8, 16],  # [1, 4, 8, 16],
        # 'fitness__alpha': [0, 0.25, 0.5, 0.75, 1], # [0, 0.1, 0.2, 0.3],  #
    },
    # {
    # #     'fitness__loss_function': [utils.fitness_log_loss],
    # #     'fitness__weight_decay': [0.0, 0.125, 0.25, 0.5, 1],  
    # #     'fitness__lr': [1e-2, 3e-3, 1e-3, 3e-4],
    # #     'fitness__k': [16, 32, 64, 128],
    # #     'fitness__batch_size': [1, 4, 8, 16],
    # # },
    # # {
    # #     'fitness__loss_function': [utils.fitness_square_square_loss],
    # #     'fitness__weight_decay': [0.0, 0.125, 0.25, 0.5, 1],  
    # #     'fitness__margin': [1, 2, 4],
    # #     'fitness__lr': [1e-2, 3e-3, 1e-3, 3e-4],
    # #     'fitness__k': [16, 32, 64, 128],
    # #     'fitness__batch_size': [1, 4, 8, 16],
    # },
    
]

all_feature_columns = [c for c in filtered_fitness_df.columns if c not in NON_FEATURE_COLUMNS]
model_kwargs = dict(output_activation=nn.Identity())
train_kwargs = dict()
cv_kwargs = dict(refit='single_game_rank')
scoring = utils.build_multiple_scoring_function(
    [utils.evaluate_fitness_overall_ecdf, utils.evaluate_fitness_single_game_rank],
    ['overall_ecdf', 'single_game_rank'],
)

cv, test_df = model_fitting_experiment(fitness_df, test_param_grid,
    scoring_function=scoring, verbose=1, 
    model_kwargs=model_kwargs, train_kwargs=train_kwargs, cv_kwargs=cv_kwargs)


Fitting 5 folds for each of 1281 candidates, totalling 6405 fits


In [35]:
cv_df = pd.concat([
    pd.DataFrame(cv.cv_results_["params"]), 
    pd.DataFrame(cv.cv_results_["mean_test_overall_ecdf"], columns=['ecdf_mean']), 
    pd.DataFrame(cv.cv_results_["std_test_overall_ecdf"], columns=['ecdf_std']), 
    pd.DataFrame(cv.cv_results_["rank_test_overall_ecdf"], columns=['ecdf_rank']),
    pd.DataFrame(cv.cv_results_["mean_test_single_game_rank"], columns=['game_rank_mean']), 
    pd.DataFrame(cv.cv_results_["std_test_single_game_rank"], columns=['game_rank_std']), 
    pd.DataFrame(cv.cv_results_["rank_test_single_game_rank"], columns=['game_rank_rank']),
],axis=1)

cv_df.sort_values(by='ecdf_rank').head(10)

,fitness__batch_size,fitness__k,fitness__loss_function,fitness__lr,fitness__margin,fitness__n_features,fitness__weight_decay,ecdf_mean,ecdf_std,ecdf_rank,game_rank_mean,game_rank_std,game_rank_rank
884,8.0,32.0,<function fitness_hinge_loss at 0x15f421120>,0.003,2.0,835,0.00,-0.350144,0.029274,1,0.854688,0.016873,1110
948,8.0,64.0,<function fitness_hinge_loss at 0x15f421120>,0.003,2.0,835,0.00,-0.353608,0.033802,2,0.853705,0.006876,1120
1207,16.0,32.0,<function fitness_hinge_loss at 0x15f421120>,0.003,2.0,835,2.00,-0.356063,0.032740,3,0.863141,0.021028,1007
885,8.0,32.0,<function fitness_hinge_loss at 0x15f421120>,0.003,2.0,835,0.25,-0.356271,0.031627,4,0.855720,0.021874,1102
944,8.0,64.0,<function fitness_hinge_loss at 0x15f421120>,0.003,1.0,835,0.00,-0.357064,0.034763,5,0.826205,0.018365,1230
830,8.0,16.0,<function fitness_hinge_loss at 0x15f421120>,0.003,8.0,835,1.00,-0.357784,0.028606,6,0.870707,0.019602,818
31,2.0,4.0,<function fitness_hinge_loss at 0x15f421120>,0.030,8.0,835,2.00,-0.357981,0.034970,7,0.872044,0.015663,755
952,8.0,64.0,<function fitness_hinge_loss at 0x15f421120>,0.003,4.0,835,0.00,-0.359118,0.031256,8,0.869749,0.016800,852
824,8.0,16.0,<function fitness_hinge_loss at 0x15f421120>,0.003,4.0,835,0.00,-0.359797,0.027583,9,0.861275,0.014540,1033
940,8.0,64.0,<function fitness_hinge_loss at 0x15f421120>,0.010,8.0,835,0.00,-0.359851,0.030216,10,0.877159,0.013203,387


In [36]:
cv_df.sort_values(by='game_rank_rank').head(10)

,fitness__batch_size,fitness__k,fitness__loss_function,fitness__lr,fitness__margin,fitness__n_features,fitness__weight_decay,ecdf_mean,ecdf_std,ecdf_rank,game_rank_mean,game_rank_std,game_rank_rank
1037,16.0,8.0,<function fitness_hinge_loss at 0x15f421120>,0.10,8.0,835,0.25,-0.370665,0.027764,520,0.881756,0.014319,1
971,16.0,4.0,<function fitness_hinge_loss at 0x15f421120>,0.10,4.0,835,2.00,-0.369820,0.024586,455,0.881373,0.013226,2
1036,16.0,8.0,<function fitness_hinge_loss at 0x15f421120>,0.10,8.0,835,0.00,-0.373218,0.022495,705,0.881312,0.014458,3
751,8.0,8.0,<function fitness_hinge_loss at 0x15f421120>,0.01,8.0,835,2.00,-0.367276,0.027111,281,0.881271,0.014042,4
275,2.0,64.0,<function fitness_hinge_loss at 0x15f421120>,0.03,1.0,835,2.00,-0.364826,0.026143,120,0.881172,0.012501,5
1111,16.0,16.0,<function fitness_hinge_loss at 0x15f421120>,0.03,2.0,835,2.00,-0.366963,0.025749,258,0.880995,0.012323,6
1263,16.0,64.0,<function fitness_hinge_loss at 0x15f421120>,0.01,8.0,835,2.00,-0.364746,0.026811,111,0.880896,0.013814,7
1100,16.0,16.0,<function fitness_hinge_loss at 0x15f421120>,0.10,8.0,835,0.00,-0.371864,0.019650,611,0.880859,0.015087,8
590,4.0,64.0,<function fitness_hinge_loss at 0x15f421120>,0.10,8.0,835,1.00,-0.368258,0.024547,343,0.880806,0.013948,9
815,8.0,16.0,<function fitness_hinge_loss at 0x15f421120>,0.01,8.0,835,2.00,-0.366391,0.025596,222,0.880752,0.015399,10


In [31]:
filtered_df = create_filtered_df(fitness_df)
feature_columns = [c for c in filtered_df.columns if c not in NON_FEATURE_COLUMNS]
full_tensor = utils.df_to_tensor(filtered_df, feature_columns)

cv.best_estimator_.fit(full_tensor)

output_path = '../models/cv_fitness_model_2022_12_26.pkl.gz'
original_output_path = output_path[:]
i = 0
while os.path.exists(output_path):
    output_path = original_output_path + f'_{i}'
    i += 1

with gzip.open(output_path, 'wb') as f:
    pickle.dump(cv.best_estimator_, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# DEFAULT_FITNESS_FUNCTION_PATH = '../models/cv_fitness_model_2022_12_23.pkl.gz'
# with gzip.open(DEFAULT_FITNESS_FUNCTION_PATH, 'rb') as f:
#     cv_fitness_model = pickle.load(f)

cv_fitness_model = cv.best_estimator_

In [ ]:
real_scores = cv_fitness_model.transform(full_tensor[:, 0, :]).detach().squeeze().numpy()
negative_scores = cv_fitness_model.transform(full_tensor[:, 1:, :]).detach().squeeze().numpy()
real_scores.mean(), real_scores.std()

In [ ]:
plt.hist(real_scores)
plt.title('Fitness energy scores of real games')
plt.xlabel('Energy score')
plt.ylabel('Count')

In [ ]:

plt.hist([real_scores, negative_scores.flatten()], stacked=True, bins=100,
    label=['Real games', 'Negatives'])
plt.title('Fitness energy scores of all games')
plt.xlabel('Energy score')
plt.ylabel('log(Count)')
plt.semilogy()
plt.legend(loc='best')

In [ ]:
weights = cv.best_estimator_.named_steps['fitness'].model.fc1.weight.data.detach().numpy().squeeze()
bias = cv.best_estimator_.named_steps['fitness'].model.fc1.bias.data.detach().numpy().squeeze()
weights.mean(), weights.std(), bias

In [ ]:
plt.hist(weights, bins=100)
plt.title('Fitness Model Weights')
plt.xlabel('Weight magnitude')
plt.ylabel('Count')

In [ ]:
n_top = 10
sorted_indices = np.argsort(weights)
top_weight_indices = sorted_indices[-n_top:]
bottom_weight_indices = sorted_indices[:n_top]

top_features = [feature_columns[i] for i in top_weight_indices][::-1]
bottom_features = [feature_columns[i] for i in bottom_weight_indices]

In [ ]:
bottom_features

In [ ]:
top_features

In [ ]:
syntethic_fitness_df = create_filtered_df(fitness_df)
feature_columns = [c for c in syntethic_fitness_df.columns if c not in NON_FEATURE_COLUMNS]
train_df, test_df = utils.train_test_split_by_game_name(syntethic_fitness_df, random_seed=42)
train_df, val_df = utils.train_test_split_by_game_name(train_df, random_seed=42)
train_tensor = utils.df_to_tensor(train_df, feature_columns)
val_tensor = utils.df_to_tensor(val_df, feature_columns)
test_tensor = utils.df_to_tensor(test_df, feature_columns)

scaler = utils.CustomSklearnScaler().fit(train_tensor)
train_tensor = scaler.transform(train_tensor)
val_tensor = scaler.transform(val_tensor)
test_tensor = scaler.transform(test_tensor)

model = utils.FitnessEenrgyModel(train_tensor.shape[-1], output_activation=nn.Identity())
model.apply(utils.init_weights)
results = utils.train_and_validate_model(
    model, train_tensor, val_tensor,
    loss_function=utils.fitness_hinge_loss,  
    loss_function_kwargs=dict(margin=4.0),
    lr=1e-2, weight_decay=0.0,
    batch_size=1, k=32,
    patience_epochs=10,
    random_seed=42,
    eval_method=utils.evaluate_fitness_flipped_sign,
)

In [ ]:
scores = model(val_tensor)
pos = scores[:, 0, None]
neg = scores[:, 1:]
pos.shape, neg.shape

In [ ]:
margin = 1
torch.relu(pos + margin - neg).shape

In [ ]:
utils.fitness_hinge_loss_with_cross_example(scores)

In [ ]:
scoring(model, train_tensor), scoring(model, val_tensor), scoring(model, test_tensor)

In [ ]:
train_preds = model(train_tensor).detach().squeeze().numpy()
val_preds = model(val_tensor).detach().squeeze().numpy()
test_preds = model(test_tensor).detach().squeeze().numpy()

In [ ]:
((train_preds[:, 0, None] < train_preds[:, 1:]).sum(axis=1) / (train_preds.shape[1] - 1)).mean()

In [ ]:
def _make_labels(prds: np.ndarray):
    labels = np.zeros_like(prds)
    labels[:, 0] = 1
    return labels

train_labels = _make_labels(train_preds)
val_labels = _make_labels(val_preds)
test_labels = _make_labels(test_preds)


In [ ]:
from sklearn import metrics


def predict_at_threshold(preds: np.ndarray, threshold: float):
    y_pred = (preds < threshold).astype(int)
    y_true = np.zeros_like(y_pred)
    y_true[:, 0] = 1
    return y_true.reshape(-1), y_pred.reshape(-1)


def evaluate_f1_with_threshold(preds: np.ndarray, threshold: float):
    return metrics.f1_score(*predict_at_threshold(preds, threshold))


def find_f1_threshold(preds: np.ndarray):
    best_f1 = 0
    best_threshold = 0
    for threshold in np.linspace(preds.min(), preds.max(), 100):
        f1 = evaluate_f1_with_threshold(preds, threshold)
        if f1 > best_f1:
            best_f1 = f1
            best_threshold = threshold

    return best_threshold, best_f1


train_threshold, train_f1 = find_f1_threshold(train_preds)
evaluate_f1_with_threshold(test_preds, train_threshold)

In [ ]:
metrics.PrecisionRecallDisplay.from_predictions(train_labels.reshape(-1), train_preds.reshape(-1))

In [ ]:
(train_preds[:, 0, np.newaxis] <= train_preds[:, 1:]).sum(axis=1)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))
ax.hist(train_preds[:, 1:].reshape(-1), bins=100, label='negatives')
ax.hist(train_preds[:, 0], bins=100, label='positives')
ax.legend()
plt.show()

In [ ]:
from statsmodels.distributions.empirical_distribution import ECDF

In [ ]:
train_ecdf = ECDF(train_preds.reshape(-1))

In [ ]:
train_ecdf(train_preds[:, 0]).mean()

In [ ]:
t = torch.tensor(train_preds)

In [ ]:
p = t[:, 0]
n = t[:, 1:]

(p[:, None] < n).mean(axis=1, dtype=torch.float)